In [7]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.neighbors import kneighbors_graph
import scipy as sc
import numpy as np

In [11]:
with open('dataset/review.json', encoding="utf8") as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings 
    # in list to actual json objects. 
    #data = list(map(json.loads, data)) 

print(data)

In [ ]:
# review_cool_id = review[['cool', 'user_id']] #.iloc
# review_cool_id

In [2]:
with open('business.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

business = pd.DataFrame(data)
business

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH
5,Richterstr. 11,"{'GoodForMeal': {'dessert': False, 'latenight'...",o9eMRCWt5PkpLDE0gOPtcQ,"[Italian, Restaurants]",Stuttgart,"{'Monday': '18:00-0:00', 'Tuesday': '18:00-0:0...",1,48.727200,9.147950,Messina,,70567,5,4.0,BW
6,"2620 Regatta Dr, Ste 102","{'BusinessAcceptsCreditCards': False, 'ByAppoi...",kCoE3jvEtg6UVz5SOD3GVw,"[Real Estate Services, Real Estate, Home Servi...",Las Vegas,"{'Friday': '8:00-17:00', 'Tuesday': '8:00-17:0...",1,36.207430,-115.268460,BDJ Realty,Summerlin,89128,5,4.0,NV
7,"7240 W Lake Mead Blvd, Ste 4","{'BusinessAcceptsCreditCards': True, 'Restaura...",OD2hnuuTJI9uotcKycxg1A,"[Shopping, Sporting Goods]",Las Vegas,"{'Monday': '11:00-19:00', 'Tuesday': '11:00-19...",1,36.197484,-115.249660,Soccer Zone,,89128,9,1.5,NV
8,2612 Brandt School Rd,"{'BusinessParking': {'garage': False, 'street'...",EsMcGiZaQuG1OOvL9iUFug,"[Coffee & Tea, Ice Cream & Frozen Yogurt, Food]",Wexford,{},1,40.615102,-80.091349,Any Given Sundae,,15090,15,5.0,PA
9,,{'BusinessAcceptsCreditCards': True},TGWhGNusxyMaA4kQVBNeew,"[Automotive, Auto Detailing]",Henderson,"{'Monday': '9:00-18:00', 'Tuesday': '9:00-18:0...",1,36.055825,-115.046350,Detailing Gone Mobile,,89014,7,5.0,NV


In [ ]:
df    = business[['latitude', 'longitude']]

In [ ]:
dfna  = df.dropna() #[pd.notnull(df)]

In [ ]:
dfna.isnull().any()

In [ ]:
x_geo = dfna.values 

In [ ]:
plt.scatter(x_geo[:,0], x_geo[:,1], s=10)
plt.show()

In [ ]:
A = kneighbors_graph(x_geo, n_neighbors=5, include_self=False)
A = A + A.T

In [ ]:
from matplotlib.collections import LineCollection

def show_graph(X, graph):
    if sc.sparse.isspmatrix(graph):
        non_zero = (sc.sparse.triu(graph, k=1) > 0)
        idx = sc.sparse.find(non_zero)
        start_idx, end_idx = idx[:2]
    else:
        non_zero = (np.triu(graph, k=1) > 0)
        start_idx, end_idx = np.where(non_zero)

    segments = np.zeros((len(start_idx), 2, 2), float)
    for i, (start,stop) in enumerate( zip(start_idx,end_idx) ):
        segments[i,:,0] = X[(start,stop),0]
        segments[i,:,1] = X[(start,stop),1]

    lc = LineCollection(segments, zorder=0)
    lc.set_linewidths(1)

    ax = plt.axes()
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_xlim(segments[:,:,0].min(), segments[:,:,0].max())
    ax.set_ylim(segments[:,:,1].min(), segments[:,:,1].max())
    ax.add_collection(lc)
    plt.show()

In [ ]:
show_graph(x_geo, A)

In [5]:
with open('tip.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. 
    # It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

tip = pd.DataFrame(data)
tip

,business_id,date,likes,text,user_id
0,tJRDll5yqpZwehenzE2cSg,2012-07-15,0,Get here early enough to have dinner.,zcTZk7OG8ovAmh_fenH21g
1,jH19V2I9fIslnNhDzPmdkA,2015-08-12,0,Great breakfast large portions and friendly wa...,ZcLKXikTHYOnYt5VYRO5sg
2,dAa0hB2yrnHzVmsCkN4YvQ,2014-06-20,0,Nice place. Great staff. A fixture in the tow...,oaYhjqBbh18ZhU0bpyzSuw
3,dAa0hB2yrnHzVmsCkN4YvQ,2016-10-12,0,Happy hour 5-7 Monday - Friday,ulQ8Nyj7jCUR8M83SUMoRQ
4,ESzO3Av0b1_TzKOiqzbQYQ,2017-01-28,0,"Parking is a premium, keep circling, you will ...",ulQ8Nyj7jCUR8M83SUMoRQ
5,k7WRPbDd7rztjHcGGkEjlw,2017-02-25,0,Homemade pasta is the best in the area,ulQ8Nyj7jCUR8M83SUMoRQ
6,k7WRPbDd7rztjHcGGkEjlw,2017-04-08,0,"Excellent service, staff is dressed profession...",ulQ8Nyj7jCUR8M83SUMoRQ
7,SqW3igh1_Png336VIb5DUA,2016-07-03,0,Come early on Sunday's to avoid the rush,ulQ8Nyj7jCUR8M83SUMoRQ
8,KNpcPGqDORDdvtekXd348w,2016-01-07,0,Love their soup!,ulQ8Nyj7jCUR8M83SUMoRQ
9,KNpcPGqDORDdvtekXd348w,2016-05-22,0,Soups are fantastic!,ulQ8Nyj7jCUR8M83SUMoRQ


In [4]:
with open('user.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

user = pd.DataFrame(data)
user

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.67,0,0,0,0,0,0,0,0,1,...,0,[],0,"[cvVMmlU1ouS3I5fhutaryQ, nj6UZ8tdGo8YJ9lUMTVWN...",0,Johnny,8,0,oMy_rEb0UBEmMlu-zcxnoQ,2014-11-03
1,3.70,0,0,0,0,0,0,0,0,0,...,0,[],0,"[0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gp...",0,Chris,10,0,JJ-aSuM4pCFPdkfoZ34q0Q,2013-09-24
2,2.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Tiffy,1,0,uUzsFQn_6cXDh6rPNGbIFA,2017-03-02
3,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Mark,6,0,mBneaEEH5EMyxaVyqS-72A,2015-03-13
4,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Evelyn,3,0,W5mJGs-dcDWRGEhAzUYtoA,2016-09-08
5,3.45,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Lisa,11,4,4E8--zUZO1Rr1IBK4_83fg,2012-07-16
6,4.78,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,B,9,0,Ob-2oGBQ7rwwYwUvhmnf7g,2012-05-01
7,5.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Peter,2,0,JaTVvKsBl0bHHJEpESn4pQ,2013-03-23
8,1.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Colleen,1,0,Ykj0DVsz0c6rX9ghjd0hDg,2010-10-10
9,4.29,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,A,7,0,kmyEPfKnHQJdTceCdoyMQg,2012-10-16
